In [1]:
!pip install transformers torch spacy
!python -m spacy download en_core_web_sm

2023-12-10 02:56:00.948809: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 02:56:00.948877: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 02:56:00.948913: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 02:56:00.958380: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 02:56:02.323391: W tensorflow/compiler/

In [28]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from torch.nn import BCEWithLogitsLoss
import spacy
import pandas as pd
import torch.nn as nn

In [4]:
class BertPreprocessor:
    def __init__(self, model_name='bert-base-uncased', max_length=256):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.max_length = max_length

    def preprocess(self, texts):
        encoding = self.tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            max_length=self.max_length,
            return_attention_mask=True,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return encoding['input_ids'], encoding['attention_mask']

In [9]:
class EntityEmbedder:
    def __init__(self, glove_file_path):
        self.embeddings_index = {}
        self.load_glove_embeddings(glove_file_path)

    def load_glove_embeddings(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                values = line.split()
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                self.embeddings_index[word] = coefs
        print(f"Loaded {len(self.embeddings_index)} word vectors.")

    def get_embedding(self, word):
        return self.embeddings_index.get(word, np.zeros(50))

    def create_embedding_matrix(self, unique_entities):
        embedding_matrix = np.zeros((len(unique_entities), 50))
        for i, entity in enumerate(unique_entities):
            embedding_matrix[i] = self.get_embedding(entity)
        return embedding_matrix

In [62]:
class BertModel:
    def __init__(self, model_name='bert-base-uncased', entity_embedding_matrix=None):
        self.model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2, output_hidden_states=True)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        if entity_embedding_matrix is not None:
            self.entity_embedding_matrix = torch.tensor(entity_embedding_matrix, dtype=torch.float).to(self.device)
            self.entity_embedding_layer = nn.Embedding.from_pretrained(self.entity_embedding_matrix)

            bert_output_size = 768
            entity_embedding_size = entity_embedding_matrix.shape[1]
            self.concatenated_layer_size = bert_output_size + entity_embedding_size

            self.classifier = nn.Linear(self.concatenated_layer_size, 2).to(self.device)

    def train(self, input_ids, attention_masks, labels, entity_indices, batch_size=16, epochs=4):
        dataset = TensorDataset(input_ids, attention_masks, labels, entity_indices)
        train_size = int(0.9 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        optimizer = AdamW(self.model.parameters(), lr=2e-5)
        loss_fct = BCEWithLogitsLoss()

        for epoch in range(epochs):
            self.model.train()
            total_loss = 0
            for batch in train_dataloader:
                b_input_ids, b_input_mask, b_labels, b_entity_indices = [t.to(self.device) for t in batch]
                self.model.zero_grad()

                outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                bert_output = outputs.hidden_states[-1][:, 0, :]

                entity_embeddings = self.entity_embedding_layer(b_entity_indices)
                entity_embeddings = torch.max(entity_embeddings, dim=1).values

                concatenated_output = torch.cat((bert_output, entity_embeddings), dim=1)
                logits = self.classifier(concatenated_output)

                b_labels = b_labels.unsqueeze(1)
                b_labels = torch.cat((1 - b_labels, b_labels), dim=1)

                loss = loss_fct(logits, b_labels.float())
                total_loss += loss.item()
                loss.backward()
                optimizer.step()

            avg_train_loss = total_loss / len(train_dataloader)
            print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_train_loss:.2f}")

            self.model.eval()
            total_eval_accuracy = 0
            for batch in val_dataloader:
                b_input_ids, b_input_mask, b_labels, b_entity_indices = [t.to(self.device) for t in batch]
                with torch.no_grad():
                    outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                    bert_output = outputs.hidden_states[-1][:, 0, :]

                    entity_embeddings = self.entity_embedding_layer(b_entity_indices)
                    entity_embeddings = torch.max(entity_embeddings, dim=1).values

                    concatenated_output = torch.cat((bert_output, entity_embeddings), dim=1)
                    logits = self.classifier(concatenated_output)

                    logits = logits.detach().cpu().numpy()
                    label_ids = b_labels.to('cpu').numpy()

                    predictions = np.argmax(logits, axis=1)
                    total_eval_accuracy += accuracy_score(label_ids, predictions)

            avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
            print(f"Validation Accuracy: {avg_val_accuracy:.2f}")


    def evaluate(self, input_ids, attention_masks, labels, entity_indices, batch_size=16):
        test_dataset = TensorDataset(input_ids, attention_masks, labels, entity_indices)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        self.model.eval()
        total_eval_accuracy = 0
        predictions, true_labels = [], []

        for batch in test_dataloader:
            b_input_ids, b_input_mask, b_labels, b_entity_indices = [t.to(self.device) for t in batch]
            with torch.no_grad():
                outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                bert_output = outputs.hidden_states[-1][:, 0, :]  # Use the last hidden state of the [CLS] token

                entity_embeddings = self.entity_embedding_layer(b_entity_indices)
                entity_embeddings = torch.max(entity_embeddings, dim=1).values  # Or use another pooling method

                concatenated_output = torch.cat((bert_output, entity_embeddings), dim=1)
                logits = self.classifier(concatenated_output)

                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()

                batch_predictions = np.argmax(logits, axis=1)
                predictions.extend(batch_predictions)
                true_labels.extend(label_ids)

        print("Accuracy:", accuracy_score(true_labels, predictions))
        print("Classification Report:")
        print(classification_report(true_labels, predictions))

In [2]:
from google.colab import files
uploaded = files.upload()


Saving glove.6B.50d.txt to glove.6B.50d.txt
Saving test_data.tsv to test_data.tsv
Saving train_data.tsv to train_data.tsv
Saving Validation_data.tsv to Validation_data.tsv


In [63]:
def main():
    # Load the dataset
    df1 = pd.read_csv('train_data.tsv', sep='\t')
    df2 = pd.read_csv('test_data.tsv', sep='\t')
    df3 = pd.read_csv('Validation_data.tsv', sep='\t')

    columns = ['index', 'id', 'label', 'statement', 'subject', 'speaker', 'JobTitle', 'State', 'Party', 'BTC', 'FC', 'HT', 'MT', 'POF', 'context', 'justification']
    df1.columns = columns
    df2.columns = columns
    df3.columns = columns

    df = pd.concat([df1, df2, df3])

    label_mapping = {'true': 1, 'half-true': 1, 'mostly-true': 1, 'false': 0, 'pants-fire': 0, 'barely-true': 0}
    df['label'] = df['label'].map(label_mapping)

    # Load GloVe embeddings and create an embedding matrix
    entity_embedder = EntityEmbedder('glove.6B.50d.txt')
    unique_entities = set(df['speaker'].tolist() + df['JobTitle'].tolist() + df['subject'].tolist() + df['Party'].tolist())
    embedding_matrix = entity_embedder.create_embedding_matrix(list(unique_entities))

    # Create a mapping from entities to their indices in the embedding matrix
    entity_to_index = {entity: i for i, entity in enumerate(unique_entities)}

    # Generate entity indices for each row in the dataset
    entity_columns = ['speaker', 'JobTitle', 'subject', 'Party']
    entity_indices = []
    for _, row in df.iterrows():
        indices = [entity_to_index.get(row[col], 0) for col in entity_columns]  # Default to 0 if entity not found
        entity_indices.append(indices)
    entity_indices = torch.tensor(entity_indices)

    # Preprocess the text data for BERT model
    bert_preprocessor = BertPreprocessor()
    texts = df['statement'].tolist()
    input_ids, attention_masks = bert_preprocessor.preprocess(texts)

    y_bert = torch.tensor(df['label'].values)

    # Initialize and train the BERT model with entity embeddings
    bert_model = BertModel('bert-base-uncased', embedding_matrix)
    bert_model.train(input_ids, attention_masks, y_bert, entity_indices)
    print("Finished Training")
    bert_model.evaluate(input_ids, attention_masks, y_bert, entity_indices)

main()

Loaded 400000 word vectors.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4 - Loss: 0.65
Validation Accuracy: 0.60
Epoch 2/4 - Loss: 0.58
Validation Accuracy: 0.61
Epoch 3/4 - Loss: 0.39
Validation Accuracy: 0.60
Epoch 4/4 - Loss: 0.16
Validation Accuracy: 0.60
Finished Training
Accuracy: 0.948310916484204
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.91      0.94      5655
           1       0.93      0.98      0.95      7133

    accuracy                           0.95     12788
   macro avg       0.95      0.94      0.95     12788
weighted avg       0.95      0.95      0.95     12788

